In [1]:
%%writefile query.hql

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2000;
SET hive.exec.max.dynamic.partitions.pernode=1000;

USE stackoverflow_;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(row_id string,
post_type_id string,
year string,
month string)
ROW FORMAT 
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = ".*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*"
)
LOCATION '/data/stackexchange1000/posts';

Overwriting query.hql


In [2]:
%%writefile -a query.hql

DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string);

Appending to query.hql


In [3]:
%%writefile -a query.hql

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*) as count, year, concat(year,"-",month) as month
WHERE year IS NOT NULL
GROUP BY year, month;

Appending to query.hql


In [4]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.124 seconds
OK
Time taken: 1.612 seconds
OK
Time taken: 0.6 seconds
OK
Time taken: 3.383 seconds
OK
Time taken: 0.297 seconds
Query ID = jovyan_20180514115555_132a6bd1-ae83-4bb7-b277-deecfc264ef8
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526254425593_0013, Tracking URL = http://731f3a71774a:8088/proxy/application_1526254425593_0013/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526254425593_0013
Hadoop job information for Stage-1: number of mappers: 1; number 

Partition stackoverflow_.posts_sample{year=2011, month=2011-11} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-12} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2012, month=2012-01} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2012, month=2012-02} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2012, month=2012-03} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2012, month=2012-04} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2012, month=2012-05} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2012, month=2012-06} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stacko

In [5]:
%%writefile final_query.hql

USE stackoverflow_;

SELECT year, month, count 
FROM (
    SELECT ROW_NUMBER() OVER(ORDER BY year, month ASC) AS row_id, year, month, count
    FROM posts_sample
) final_table
WHERE row_id=3;

Overwriting final_query.hql


In [6]:
%%bash
hive -f final_query.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.148 seconds
Query ID = jovyan_20180514115757_47883404-12fa-4c04-844c-b229abb974d4
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526254425593_0014, Tracking URL = http://731f3a71774a:8088/proxy/application_1526254425593_0014/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526254425593_0014
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-14 11:57:32,076 Stage-1 map = 0%,  reduce = 0%
2018-05-14 11:57:40,928 Stage-1 map = 100%, 